In [ ]:
import scipy.io as sio
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

pathToData = 'data/rit18.mat'
mat_contents = sio.loadmat(pathToData)

train_data = mat_contents['train_data'].astype('int16',casting ='same_kind')
train_labels = mat_contents['train_labels'].astype('int16',casting ='same_kind')